In [1]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

2023-09-15 07:02:10.851510: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-15 07:02:11.814126: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [3]:
subject = 'Maranhão - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [4]:
data = pd.read_csv('../Inputs Atuais/2008_01_model_input_MA.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data = data.drop('Maranhão - Produção de Cimento (t)', axis=1)
data

,Ano,Maranhão - IDH Educacao,Maranhão - Desemprego,Maranhão - IDH Longevidade,Maranhão - PIB - Estadual,Maranhão - PIB - Construção Civil,Maranhão - PIB - Per Capita,Maranhão - PIB - Preços de Mercado,Maranhão - IDH Renda,Contratação Comercial (Bi R$),...,Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Maranhão - IDH,Maranhão - Consumo de Cimento (t)
0,2008,0.645315,7.978325,0.736322,4.598085e+07,2.691136e+06,6.129297,3.956333e+07,0.592503,5.483335e+08,...,11.760983,25.354093,1004.987958,5445.077556,26.913732,23.175592,80.950690,1.958593,0.656244,963.205000
1,2009,0.650266,8.007891,0.737873,4.630053e+07,2.690232e+06,6.126304,3.956853e+07,0.594639,5.302764e+08,...,9.472585,28.506238,1004.900392,3620.312713,26.992836,23.414460,81.602937,1.847599,0.658950,1123.586000
2,2010,0.654819,7.971707,0.739020,4.751040e+07,2.700684e+06,6.153755,3.988864e+07,0.596141,5.046425e+08,...,9.332485,31.838993,1004.891838,2594.109885,26.968458,23.782363,83.292728,1.860782,0.661204,1378.741000
3,2011,0.658970,7.961524,0.739772,4.866181e+07,2.701549e+06,6.182820,4.018069e+07,0.597021,4.854622e+08,...,10.985689,33.196582,1005.053412,2035.008505,27.022388,24.015667,84.183211,1.989042,0.663017,1282.016000
4,2012,0.662311,10.302202,0.740034,4.972690e+07,2.696497e+06,6.208137,4.043865e+07,0.597229,4.867661e+08,...,8.159014,31.592321,1005.214749,1723.384057,26.989407,23.760558,83.318460,1.993051,0.664173,1528.770000
5,2013,0.666586,11.189661,0.740120,5.062157e+07,2.675613e+06,6.222354,4.060687e+07,0.596832,5.225848e+08,...,7.913561,29.227149,1005.449976,1548.437409,26.912712,23.364264,81.867671,1.963286,0.665521,1563.125000
6,2014,0.668885,11.986521,0.739796,5.132137e+07,2.651392e+06,6.227127,4.069030e+07,0.595984,5.676413e+08,...,10.350442,28.491261,1005.705374,1452.214268,27.001638,23.279428,81.037312,1.988340,0.665898,1303.411571
7,2015,0.671376,12.529442,0.739029,5.174856e+07,2.602821e+06,6.218491,4.066452e+07,0.594816,5.179392e+08,...,12.454398,29.089818,1005.751406,1363.412394,27.085524,23.327394,80.918789,1.953057,0.666139,1191.509657
8,2016,0.688469,12.762358,0.746544,5.210281e+07,2.570101e+06,6.220890,4.069623e+07,0.606351,4.524144e+08,...,13.111908,28.880429,1005.704586,1297.690355,27.029688,23.289212,81.028777,1.867218,0.677976,1100.217771
9,2017,0.689875,12.460135,0.745411,5.221165e+07,2.559679e+06,6.221665,4.069725e+07,0.605565,4.213689e+08,...,9.479812,27.965594,1005.796545,1279.407347,26.893536,23.259717,81.515345,1.794104,0.677792,971.921000


In [5]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Maranhão - IDH Educacao,Maranhão - Desemprego,Maranhão - IDH Longevidade,Maranhão - PIB - Estadual,Maranhão - PIB - Construção Civil,Maranhão - PIB - Per Capita,Maranhão - PIB - Preços de Mercado,Maranhão - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Maranhão - IDH
0,0.645315,7.978325,0.736322,4.598085e+07,2.691136e+06,6.129297,3.956333e+07,0.592503,5.483335e+08,2.546644e+09,...,2.868108,11.760983,25.354093,1004.987958,5445.077556,26.913732,23.175592,80.950690,1.958593,0.656244
1,0.650266,8.007891,0.737873,4.630053e+07,2.690232e+06,6.126304,3.956853e+07,0.594639,5.302764e+08,2.679945e+09,...,2.305885,9.472585,28.506238,1004.900392,3620.312713,26.992836,23.414460,81.602937,1.847599,0.658950
2,0.654819,7.971707,0.739020,4.751040e+07,2.700684e+06,6.153755,3.988864e+07,0.596141,5.046425e+08,2.891930e+09,...,1.958246,9.332485,31.838993,1004.891838,2594.109885,26.968458,23.782363,83.292728,1.860782,0.661204
3,0.658970,7.961524,0.739772,4.866181e+07,2.701549e+06,6.182820,4.018069e+07,0.597021,4.854622e+08,3.091988e+09,...,2.079088,10.985689,33.196582,1005.053412,2035.008505,27.022388,24.015667,84.183211,1.989042,0.663017
4,0.662311,10.302202,0.740034,4.972690e+07,2.696497e+06,6.208137,4.043865e+07,0.597229,4.867661e+08,3.189506e+09,...,2.183282,8.159014,31.592321,1005.214749,1723.384057,26.989407,23.760558,83.318460,1.993051,0.664173
5,0.666586,11.189661,0.740120,5.062157e+07,2.675613e+06,6.222354,4.060687e+07,0.596832,5.225848e+08,3.093872e+09,...,2.403445,7.913561,29.227149,1005.449976,1548.437409,26.912712,23.364264,81.867671,1.963286,0.665521
6,0.668885,11.986521,0.739796,5.132137e+07,2.651392e+06,6.227127,4.069030e+07,0.595984,5.676413e+08,2.779340e+09,...,2.748397,10.350442,28.491261,1005.705374,1452.214268,27.001638,23.279428,81.037312,1.988340,0.665898
7,0.671376,12.529442,0.739029,5.174856e+07,2.602821e+06,6.218491,4.066452e+07,0.594816,5.179392e+08,2.561731e+09,...,2.501529,12.454398,29.089818,1005.751406,1363.412394,27.085524,23.327394,80.918789,1.953057,0.666139
8,0.688469,12.762358,0.746544,5.210281e+07,2.570101e+06,6.220890,4.069623e+07,0.606351,4.524144e+08,2.516859e+09,...,2.073464,13.111908,28.880429,1005.704586,1297.690355,27.029688,23.289212,81.028777,1.867218,0.677976
9,0.689875,12.460135,0.745411,5.221165e+07,2.559679e+06,6.221665,4.069725e+07,0.605565,4.213689e+08,2.497710e+09,...,2.177386,9.479812,27.965594,1005.796545,1279.407347,26.893536,23.259717,81.515345,1.794104,0.677792


In [6]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     1123.586000
1     1378.741000
2     1282.016000
3     1528.770000
4     1563.125000
5     1303.411571
6     1191.509657
7     1100.217771
8      971.921000
9      750.012000
10     774.045000
11    1038.249000
12    1059.129000
13    1083.999000
14            NaN
Name: Maranhão - Consumo de Cimento (t), dtype: float64

In [7]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Maranhão - IDH Educacao,Maranhão - Desemprego,Maranhão - IDH Longevidade,Maranhão - PIB - Estadual,Maranhão - PIB - Construção Civil,Maranhão - PIB - Per Capita,Maranhão - PIB - Preços de Mercado,Maranhão - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Maranhão - IDH
0,-1.548004,-1.341931,-1.526781,-1.800669,0.888019,-1.813414,-1.863562,-1.353287,1.204752,-0.990234,...,2.026352,0.984445,-1.704426,-1.179358,2.780896,-0.581090,-1.106443,-1.032012,0.713734,-1.515780
1,-1.235625,-1.327021,-0.992611,-1.657451,0.872218,-1.896902,-1.851185,-0.893854,0.820657,-0.546688,...,0.085558,-0.051710,-0.236979,-1.419199,1.260440,0.474888,-0.140579,-0.401753,-0.347981,-1.152490
2,-0.948330,-1.345268,-0.597544,-1.115428,1.054815,-1.131157,-1.089628,-0.570965,0.275392,0.158672,...,-1.114490,-0.115145,1.314548,-1.442629,0.405374,0.149463,1.347032,1.231074,-0.221880,-0.849804
3,-0.686418,-1.350403,-0.338594,-0.599596,1.069928,-0.320349,-0.394850,-0.381737,-0.132596,0.824346,...,-0.697344,0.633403,1.946559,-1.000083,-0.060488,0.869373,2.290396,2.091538,1.004993,-0.606270
4,-0.475631,-0.170051,-0.248616,-0.122433,0.981661,0.385888,0.218857,-0.336916,-0.104860,1.148829,...,-0.337667,-0.646476,1.199713,-0.558185,-0.320144,0.429102,1.258865,1.255939,1.043345,-0.450985
5,-0.205832,0.277475,-0.218954,0.278382,0.616846,0.782503,0.619058,-0.422410,0.657046,0.830615,...,0.422337,-0.757614,0.098633,0.086092,-0.465916,-0.594704,-0.343548,-0.145943,0.758629,-0.269993
6,-0.060778,0.679314,-0.330312,0.591894,0.193718,0.915640,0.817536,-0.604630,1.615453,-0.215960,...,1.613108,0.345772,-0.243951,0.785621,-0.546092,0.592379,-0.686579,-0.948310,0.998286,-0.219397
7,0.096364,0.953097,-0.594440,0.783275,-0.654791,0.674736,0.756208,-0.855798,0.558230,-0.940033,...,0.760923,1.298413,0.034700,0.911699,-0.620085,1.712186,-0.492629,-1.062838,0.660786,-0.186967
8,1.174906,1.070551,1.993279,0.941980,-1.226398,0.741645,0.831633,1.624690,-0.835563,-1.089341,...,-0.716758,1.596124,-0.062778,0.783463,-0.674847,0.966821,-0.647020,-0.956557,-0.160319,1.402510
9,1.263639,0.918147,1.603303,0.990742,-1.408452,0.763263,0.834059,1.455606,-1.495939,-1.153056,...,-0.358019,-0.048438,-0.488670,1.035335,-0.690081,-0.850683,-0.766284,-0.486392,-0.859692,1.377796


In [8]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0     1123.586000
1     1378.741000
2     1282.016000
3     1528.770000
4     1563.125000
5     1303.411571
6     1191.509657
7     1100.217771
8      971.921000
9      750.012000
10     774.045000
11    1038.249000
Name: Maranhão - Consumo de Cimento (t), dtype: float64

In [9]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Maranhão - IDH Educacao,Maranhão - Desemprego,Maranhão - IDH Longevidade,Maranhão - PIB - Estadual,Maranhão - PIB - Construção Civil,Maranhão - PIB - Per Capita,Maranhão - PIB - Preços de Mercado,Maranhão - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Maranhão - IDH
12,1.137321,0.665597,-0.713268,0.506899,-0.617267,0.098822,0.211599,0.829595,1.108405,2.635823,...,-1.482779,-2.260447,-0.619583,0.376717,-0.483958,-1.82319,-0.372439,0.371831,-1.374154,0.896376


In [10]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    1059.129
Name: Maranhão - Consumo de Cimento (t), dtype: float64

In [12]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(2048, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(512, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-1:], train_target.iloc[-1:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [13]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [14]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[3420660484, 1456259290, 873964471, 263084997, 2899017490, 2892309360, 4124708540, 2274105689, 1695854363, 94815506, 975362137, 973248272, 1328404417, 2854438300, 422912773, 2843837679, 3670144857, 2824770172, 2092382079, 2658536633, 2902379852, 95106992, 4166402362, 3608268977, 623625063, 2749187518, 879323200, 3994620710, 985060668, 3087426061, 2588100649, 1794206222, 314448554, 1739294933, 3845703452, 2348458375, 833618162, 3213731783, 541747298, 2906292212, 1268204620, 2005612303, 1780979285, 2797505465, 3834508928, 3584437033, 4244868957, 585030800, 625323826, 2610543710, 1084608507, 1909209785, 1251706715, 2415208121, 2876087119, 3966936023, 2457153434, 1646387449, 2690030680, 183691481, 1920785277, 2982481085, 429926523, 1190782459, 3400446508, 3415751072, 3554020873, 2683313314, 1843818442, 626201455, 3190539215, 2181218127, 4006298179, 3892028576, 1564791572, 3670309411, 2500738610, 2719369532, 630172637, 1958747095, 2136207717, 1788503390, 2263595082, 2489760702, 1279177518, 

2023-09-15 07:02:39.047521: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-15 07:02:39.275620: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-15 07:02:39.275780: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

val_loss: 3094.94189453125
winner_seed: 3420660484


Step: 1 ___________________________________________
val_loss: 0.2970728874206543
winner_seed: 1456259290


Step: 2 ___________________________________________
val_loss: 3785.734130859375


Step: 3 ___________________________________________
val_loss: 0.6833670139312744


Step: 4 ___________________________________________
val_loss: 3400.931396484375


Step: 5 ___________________________________________
val_loss: 3584.204345703125


Step: 6 ___________________________________________
val_loss: 3871.780517578125


Step: 7 ___________________________________________
val_loss: 17255.4453125


Step: 8 ___________________________________________
val_loss: 3013.945556640625


Step: 9 ___________________________________________
val_loss: 17843.212890625


Step: 10 ___________________________________________
val_loss: 1907.6805419921875


Step: 11 ___________________________________________
val_loss: 10880.970703125


Step: 12 _______________

val_loss: 3990.099853515625


final_seed: 985060668


In [15]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 1s 538ms/step - loss: 1424572.6250 - val_loss: 1112516.3750
Epoch 2/10000
1/1 [==============================] - 0s 15ms/step - loss: 1346040.8750 - val_loss: 1231236.5000
Epoch 3/10000
1/1 [==============================] - 0s 14ms/step - loss: 1270359.3750 - val_loss: 1657228.7500
Epoch 4/10000
1/1 [==============================] - 0s 14ms/step - loss: 1098005.6250 - val_loss: 2475201.5000
Epoch 5/10000
1/1 [==============================] - 0s 14ms/step - loss: 1035220.1875 - val_loss: 2271655.0000
Epoch 6/10000
1/1 [==============================] - 0s 14ms/step - loss: 1156769.6250 - val_loss: 1556269.0000
Epoch 7/10000
1/1 [==============================] - 0s 14ms/step - loss: 1249000.8750 - val_loss: 2009918.8750
Epoch 8/10000
1/1 [==============================] - 0s 14ms/step - loss: 959159.4375 - val_loss: 2167255.5000
Epoch 9/10000
1/1 [==============================] - 0s 14ms/step - loss: 1130607.6250 - val_loss: 20712

1/1 [==============================] - 0s 15ms/step - loss: 410374.8750 - val_loss: 329219.4688
Epoch 75/10000
1/1 [==============================] - 0s 15ms/step - loss: 369197.0938 - val_loss: 235017.5312
Epoch 76/10000
1/1 [==============================] - 0s 14ms/step - loss: 267629.1250 - val_loss: 255004.0469
Epoch 77/10000
1/1 [==============================] - 0s 14ms/step - loss: 333899.8438 - val_loss: 310524.9688
Epoch 78/10000
1/1 [==============================] - 0s 14ms/step - loss: 285089.0000 - val_loss: 296930.8125
Epoch 79/10000
1/1 [==============================] - 0s 14ms/step - loss: 317822.0938 - val_loss: 248964.1562
Epoch 80/10000
1/1 [==============================] - 0s 14ms/step - loss: 446554.5938 - val_loss: 96508.9766
Epoch 81/10000
1/1 [==============================] - 0s 14ms/step - loss: 325753.4375 - val_loss: 226956.4531
Epoch 82/10000
1/1 [==============================] - 0s 14ms/step - loss: 269818.0312 - val_loss: 233762.4531
Epoch 83/10000
1/

Epoch 148/10000
1/1 [==============================] - 0s 13ms/step - loss: 157811.0781 - val_loss: 17663.3730
Epoch 149/10000
1/1 [==============================] - 0s 14ms/step - loss: 97783.1953 - val_loss: 56298.9141
Epoch 150/10000
1/1 [==============================] - 0s 14ms/step - loss: 105461.6250 - val_loss: 53841.8711
Epoch 151/10000
1/1 [==============================] - 0s 14ms/step - loss: 127929.0938 - val_loss: 54899.4570
Epoch 152/10000
1/1 [==============================] - 0s 14ms/step - loss: 101743.1016 - val_loss: 40159.7773
Epoch 153/10000
1/1 [==============================] - 0s 14ms/step - loss: 117825.2109 - val_loss: 59678.9062
Epoch 154/10000
1/1 [==============================] - 0s 14ms/step - loss: 92956.1875 - val_loss: 63735.3555
Epoch 155/10000
1/1 [==============================] - 0s 13ms/step - loss: 124055.6016 - val_loss: 54174.1250
Epoch 156/10000
1/1 [==============================] - 0s 14ms/step - loss: 71841.2656 - val_loss: 67909.1641
Epoc

1/1 [==============================] - 0s 14ms/step - loss: 102526.8672 - val_loss: 5363.2734
Epoch 223/10000
1/1 [==============================] - 0s 14ms/step - loss: 68586.8984 - val_loss: 22.5103
Epoch 224/10000
1/1 [==============================] - 0s 14ms/step - loss: 90337.7188 - val_loss: 7321.4741
Epoch 225/10000
1/1 [==============================] - 0s 14ms/step - loss: 51836.1133 - val_loss: 848.5607
Epoch 226/10000
1/1 [==============================] - 0s 14ms/step - loss: 38713.3672 - val_loss: 3852.9055
Epoch 227/10000
1/1 [==============================] - 0s 14ms/step - loss: 74128.5000 - val_loss: 2027.3793
Epoch 228/10000
1/1 [==============================] - 0s 14ms/step - loss: 43087.9844 - val_loss: 6353.3374
Epoch 229/10000
1/1 [==============================] - 0s 14ms/step - loss: 43111.4375 - val_loss: 4948.4404
Epoch 230/10000
1/1 [==============================] - 0s 14ms/step - loss: 65175.0430 - val_loss: 1236.5972
Epoch 231/10000
1/1 [================

1/1 [==============================] - 0s 14ms/step - loss: 60031.8242 - val_loss: 15020.8184
Epoch 298/10000
1/1 [==============================] - 0s 15ms/step - loss: 65086.9492 - val_loss: 10510.6543
Epoch 299/10000
1/1 [==============================] - 0s 14ms/step - loss: 51274.3945 - val_loss: 1261.2729
Epoch 300/10000
1/1 [==============================] - 0s 14ms/step - loss: 66439.2891 - val_loss: 3146.7004
Epoch 301/10000
1/1 [==============================] - 0s 14ms/step - loss: 62608.4219 - val_loss: 1639.2218
Epoch 302/10000
1/1 [==============================] - 0s 14ms/step - loss: 59366.1445 - val_loss: 3757.8906
Epoch 303/10000
1/1 [==============================] - 0s 14ms/step - loss: 54891.3750 - val_loss: 2506.3394
Epoch 304/10000
1/1 [==============================] - 0s 13ms/step - loss: 67029.4922 - val_loss: 242.8187
Epoch 305/10000
1/1 [==============================] - 0s 14ms/step - loss: 33817.9648 - val_loss: 0.0918
Epoch 306/10000
1/1 [================

...

1/1 [==============================] - 0s 15ms/step - loss: 22532.0469 - val_loss: 16804.4766
Epoch 672/10000
1/1 [==============================] - 0s 15ms/step - loss: 20782.8320 - val_loss: 17857.0918
Epoch 673/10000
1/1 [==============================] - 0s 14ms/step - loss: 17718.0703 - val_loss: 30102.0391
Epoch 674/10000
1/1 [==============================] - 0s 14ms/step - loss: 22480.9160 - val_loss: 19377.3574
Epoch 675/10000
1/1 [==============================] - 0s 14ms/step - loss: 18788.9004 - val_loss: 24609.3066
Epoch 676/10000
1/1 [==============================] - 0s 14ms/step - loss: 20473.1816 - val_loss: 17031.4121
Epoch 677/10000
1/1 [==============================] - 0s 13ms/step - loss: 20096.1855 - val_loss: 26122.1680
Epoch 678/10000
1/1 [==============================] - 0s 14ms/step - loss: 22008.1934 - val_loss: 18201.6387
Epoch 679/10000
1/1 [==============================] - 0s 14ms/step - loss: 21685.6777 - val_loss: 17032.9590
Epoch 680/10000
1/1 [=====

1/1 [==============================] - 0s 14ms/step - loss: 22109.7500 - val_loss: 3590.2800
Epoch 746/10000
1/1 [==============================] - 0s 14ms/step - loss: 25625.5293 - val_loss: 2393.4873
Epoch 747/10000
1/1 [==============================] - 0s 14ms/step - loss: 22984.5312 - val_loss: 3757.8083
Epoch 748/10000
1/1 [==============================] - 0s 14ms/step - loss: 19502.0234 - val_loss: 4975.3975
Epoch 749/10000
1/1 [==============================] - 0s 14ms/step - loss: 32654.9980 - val_loss: 5850.9707
Epoch 750/10000
1/1 [==============================] - 0s 14ms/step - loss: 24425.4062 - val_loss: 2671.0552
Epoch 751/10000
1/1 [==============================] - 0s 14ms/step - loss: 21842.0781 - val_loss: 3961.6360
Epoch 752/10000
1/1 [==============================] - 0s 14ms/step - loss: 22606.3184 - val_loss: 5069.9199
Epoch 753/10000
1/1 [==============================] - 0s 14ms/step - loss: 22242.0547 - val_loss: 4804.6118
Epoch 754/10000
1/1 [==============

Epoch 820/10000
1/1 [==============================] - 0s 14ms/step - loss: 20658.9453 - val_loss: 10380.7539
Epoch 821/10000
1/1 [==============================] - 0s 14ms/step - loss: 28455.9277 - val_loss: 21590.0371
Epoch 822/10000
1/1 [==============================] - 0s 14ms/step - loss: 22142.0156 - val_loss: 12337.0479
Epoch 823/10000
1/1 [==============================] - 0s 14ms/step - loss: 20345.9863 - val_loss: 17259.9199
Epoch 824/10000
1/1 [==============================] - 0s 15ms/step - loss: 21127.9707 - val_loss: 40319.8516
Epoch 825/10000
1/1 [==============================] - 0s 15ms/step - loss: 18068.2637 - val_loss: 26243.0117
Epoch 826/10000
1/1 [==============================] - 0s 14ms/step - loss: 18808.8145 - val_loss: 27090.2578
Epoch 827/10000
1/1 [==============================] - 0s 15ms/step - loss: 21399.3379 - val_loss: 28554.2344
Epoch 828/10000
1/1 [==============================] - 0s 14ms/step - loss: 33353.7305 - val_loss: 13517.8096
Epoch 829/

1/1 [==============================] - 0s 14ms/step - loss: 21733.7598 - val_loss: 21737.7988
Epoch 895/10000
1/1 [==============================] - 0s 14ms/step - loss: 25134.3965 - val_loss: 19220.4453
Epoch 896/10000
1/1 [==============================] - 0s 14ms/step - loss: 18923.4062 - val_loss: 21607.9766
Epoch 897/10000
1/1 [==============================] - 0s 14ms/step - loss: 22637.7031 - val_loss: 15919.7734
Epoch 898/10000
1/1 [==============================] - 0s 14ms/step - loss: 19542.3535 - val_loss: 26259.4277
Epoch 899/10000
1/1 [==============================] - 0s 14ms/step - loss: 21522.7812 - val_loss: 19882.2734
Epoch 900/10000
1/1 [==============================] - 0s 14ms/step - loss: 17566.6270 - val_loss: 28000.5391
Epoch 901/10000
1/1 [==============================] - 0s 13ms/step - loss: 23383.1582 - val_loss: 21464.5742
Epoch 902/10000
1/1 [==============================] - 0s 13ms/step - loss: 20300.1270 - val_loss: 20054.8047
Epoch 903/10000
1/1 [=====

1/1 [==============================] - 0s 14ms/step - loss: 22088.2734 - val_loss: 20116.0137
Epoch 969/10000
1/1 [==============================] - 0s 14ms/step - loss: 22351.5879 - val_loss: 21318.6914
Epoch 970/10000
1/1 [==============================] - 0s 14ms/step - loss: 20004.7676 - val_loss: 19646.5469
Epoch 971/10000
1/1 [==============================] - 0s 14ms/step - loss: 20888.1875 - val_loss: 19405.0137
Epoch 972/10000
1/1 [==============================] - 0s 14ms/step - loss: 25798.9941 - val_loss: 24431.1934
Epoch 973/10000
1/1 [==============================] - 0s 14ms/step - loss: 17194.4512 - val_loss: 26369.2871
Epoch 974/10000
1/1 [==============================] - 0s 13ms/step - loss: 18504.5254 - val_loss: 23701.1133
Epoch 975/10000
1/1 [==============================] - 0s 13ms/step - loss: 20266.7480 - val_loss: 23414.3418
Epoch 976/10000
1/1 [==============================] - 0s 14ms/step - loss: 21898.6465 - val_loss: 17272.2539
Epoch 977/10000
1/1 [=====

In [16]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 50ms/step
[1059.129](test_target) - [[1038.3705]](prediction) = 20.758516601562405


In [17]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [18]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Maranhão - IDH Educacao,Maranhão - Desemprego,Maranhão - IDH Longevidade,Maranhão - PIB - Estadual,Maranhão - PIB - Construção Civil,Maranhão - PIB - Per Capita,Maranhão - PIB - Preços de Mercado,Maranhão - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Maranhão - IDH
0,-1.241459,-0.486246,-1.281082,-0.935735,-0.609310,-0.581902,-0.724135,-1.289469,1.148205,-1.122043,...,1.308949,1.412346,-1.213218,1.409635,1.321922,-1.348681,-1.129447,-1.011331,1.405833,-1.260158
1,0.034142,1.393198,0.121772,-0.450446,-0.800585,-0.825312,-0.689940,0.141790,0.140893,-0.184474,...,-0.190804,-0.643250,-0.022737,-0.606347,-0.225769,1.042844,-0.172339,-0.350435,-0.836052,0.074200
2,1.207317,-0.906952,1.159310,1.386181,1.409895,1.407214,1.414076,1.147679,-1.289098,1.306518,...,-1.118144,-0.769096,1.235955,-0.803288,-1.096153,0.305837,1.301786,1.361766,-0.569781,1.185958


train_target:


,Maranhão - Consumo de Cimento (t)
0,1123.586
1,1378.741
2,1282.016


test_input:


...

,Maranhão - IDH Educacao,Maranhão - Desemprego,Maranhão - IDH Longevidade,Maranhão - PIB - Estadual,Maranhão - PIB - Construção Civil,Maranhão - PIB - Per Capita,Maranhão - PIB - Preços de Mercado,Maranhão - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Maranhão - IDH
12,1.137321,0.665597,-0.713268,0.506899,-0.617267,0.098822,0.211599,0.829595,1.108405,2.635823,...,-1.482779,-2.260447,-0.619583,0.376717,-0.483958,-1.82319,-0.372439,0.371831,-1.374154,0.896376


test_target:


,Maranhão - Consumo de Cimento (t)
12,1059.129


1/1 [==============================] - 0s 39ms/step
Error: 20.758516601562405




[1282.007080078125,
 1405.386962890625,
 1748.5399169921875,
 1455.9046630859375,
 1191.8665771484375,
 1100.03662109375,
 1069.2335205078125,
 978.7909545898438,
 915.1029663085938,
 1038.3704833984375]

In [19]:
display(targets)
display(predictions)

[1528.77,
 1563.125,
 1303.4115714285715,
 1191.5096571428571,
 1100.2177714285715,
 971.921,
 750.012,
 774.045,
 1038.249,
 1059.129]

[1282.007080078125,
 1405.386962890625,
 1748.5399169921875,
 1455.9046630859375,
 1191.8665771484375,
 1100.03662109375,
 1069.2335205078125,
 978.7909545898438,
 915.1029663085938,
 1038.3704833984375]

In [20]:
mae = mean_absolute_error(predictions, targets)
mae

200.16607607421872

In [21]:
porcentage = mae/np.mean(targets)
porcentage

0.1774460599981195